# MELODI Presto Use Cases

In [4]:
import json
import pandas as pd
import requests
import matplotlib.pyplot as plt
%matplotlib inline 
import numpy as np
from utils import enrich, overlap, sentence

## Configure parameters

In [3]:
API_URL = "https://melodi-presto.mrcieu.ac.uk/api/"

requests.get(f"{API_URL}/status").json()

True

# Run the overlap function on some examples

### Genetic basis of psoriasis

https://www.cell.com/ajhg/fulltext/S0002-9297(12)00157-7

In [5]:
q1=['AP1S3','IL36RN','CARD14']
q2=['Psoriasis']

overlap_df = overlap(q1,q2)

In [6]:
overlap_counts = overlap_df.groupby(['set_x','object_type_x'])['object_name_x'].value_counts()
overlap_counts

set_x   object_type_x  object_name_x                
ap1s3   gngm           CARD14                             5
card14  aapp           NF-kappa B                         3
        dsyn           Psoriasis                        861
                       Autoimmune Diseases               10
                       skin disorder                      8
        gngm           NF-kappa B                        21
                       CARD14                             5
il36rn  aapp           Interleukin Receptor               1
        dsyn           Arthritis, Psoriatic              33
                       Pustulosis of Palms and Soles      3
        gngm           CARD14                             5
                       Interleukin Receptor               1
Name: object_name_x, dtype: int64

### Drug repurposing

https://www.medrxiv.org/content/10.1101/2020.05.07.20093286v1

In [7]:
q1=['DHODH', 'ITGB5', 'JAK2']
q2=['Leflunomide','Cilengitide','Baricitinib']

overlap_df = overlap(q1,q2)

In [8]:
overlap_counts = overlap_df.groupby(['set_x','set_y','object_type_x'])['object_name_x'].value_counts()
overlap_counts

set_x  set_y        object_type_x  object_name_x                     
dhodh  leflunomide  aapp           Phosphotransferases                    1
                    dsyn           Rheumatoid Arthritis                   9
                    gngm           Dihydroorotate dehydrogenase          50
                                   Dihydroorotate dehydrogenase|DHODH     1
                    orch           leflunomide                           92
                                   Pyrimidine                            12
jak2   baricitinib  aapp           Janus kinase                           4
                                   Janus kinase 1|JAK1                    4
                                   cytokine                               3
                    gngm           Janus kinase                          11
                                   Janus kinase 1|JAK1                    4
                                   cytokine                               1
                  

### Obesity and thyroid cancer

https://academic.oup.com/jcem/article/105/7/dgaa250/5835841

In [15]:
q1=['obesity']
q2=['thyroid cancer']

overlap_df = overlap(q1,q2)
overlap_counts = overlap_df.groupby(['object_type_x'])['object_name_x'].value_counts()
overlap_counts

object_type_x  object_name_x                     
aapp           ghrelin                               10
               PPAR gamma                             5
               Proto-Oncogene Proteins c-akt|AKT1     4
               receptor                               3
               FRAP1 protein, human|MTOR              1
               Sex Hormone-Binding Globulin           1
               Somatostatin                           1
dsyn           Syndrome                              26
               Crohn's disease                        2
               Hypothyroidism                         2
               Thrombophilia                          2
               Thrombus                               1
gngm           Proto-Oncogene Proteins c-akt|AKT1     4
               PPAR gamma                             3
               ghrelin                                3
               Sex Hormone-Binding Globulin           2
               MAPKAP1                                

### Coronavirus and dexamethasone

Recent work (https://www.recoverytrial.net/) has demonstrated a potential beneficial effect of dexamethasone on covid-19. Here we can explore the potential intermediates connecting them, including genes, diseases and hormones. We can also separate the two terms `coronavirus` and `covid-19` to distinguish which semantic terms are associated with each query. 

In [16]:
q1=['dexamethasone']
q2=['coronavirus','covid-19']

overlap_df = overlap(q1,q2)
overlap_counts = overlap_df.groupby(['set_y','object_type_x'])['object_name_x'].value_counts()
overlap_counts

set_y        object_type_x  object_name_x                            
coronavirus  aapp           cytokine                                     12
                            receptor                                     10
                            TNF protein, human|TNF                        6
                            Peptides                                      2
                            Pulmonary Surfactant-Associated Protein A     1
                            Pulmonary Surfactant-Associated Protein D     1
             dsyn           Infection                                    11
                            Pneumonia                                    10
                            Diabetes                                      4
                            Hypertensive disease                          4
                            Syndrome                                      4
                            Respiratory Distress Syndrome, Adult          3
                  

# Identifying risk factors for a disease

We can explore one disease in detail to identify risk factors. In this case, `psoriasis`.

In [63]:
q='psoriasis'
enrich_df=enrich(q)

#map to objects that contain the query term
enrich_df = enrich_df[enrich_df['object_name'].str.contains(q,case=False)]

#list of risk factor predicates
rf_preds=['CAUSES','PREDISPOSES','PRECEDES','STIMULATES']
rf=enrich_df[enrich_df['predicate'].isin(rf_preds)]

#make sure pval is a float
rf['pval']=rf['pval'].astype(float)

#look at the top 20 ordered by enrichment pvalue
rf.sort_values(by='pval',ascending=True).head(20)[['subject_name','subject_type','predicate','object_name','pval','localCount']]

/Users/be15516/anaconda3/envs/MelodiPresto/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,subject_name,subject_type,predicate,object_name,pval,localCount
0,imiquimod,orch,CAUSES,Psoriasis,0.000000e+00,401
9,cytokine,aapp,CAUSES,Psoriasis,3.864715e-117,66
21,interleukin-23,gngm,CAUSES,Psoriasis,1.574232e-78,44
27,"TNF protein, human|TNF",aapp,CAUSES,Psoriasis,4.822940e-70,39
37,Interleukin-17,aapp,CAUSES,Psoriasis,3.140959e-54,30
38,infliximab,aapp,CAUSES,Psoriasis,6.153936e-54,30
84,Tumor Necrosis Factor-alpha|TNF,aapp,CAUSES,Psoriasis,4.117160e-33,18
92,Obesity,dsyn,PREDISPOSES,Psoriasis,2.384463e-31,17
128,TNF gene|TNF,gngm,CAUSES,Psoriasis,4.682245e-26,14
191,tumor necrosis factor-alpha inhibitor,aapp,CAUSES,Psoriasis,9.401904e-21,11


# All against all

The excellent performance capabilities of MELODI Presto means we can perform an all-against-all analysis for a range of terms, e.g. a list of genes and diseases. This can create a network, highlighting potential shared mechanisms of action. 

As an example we examine the relationships between 

In [17]:
#just run enrich on each then create network